# Depth from video

In [ ]:
########################################################################################
# Author: Justin Clancy, University of Melbourne, 2020                                 #
# - Extension Project -                                                                #
# Creating Depth Maps from Stereo Video                                                #
# + Camera Calibration Functions (not explicitly used but written)                     #
# Note: functions were ran with less than optimal equipment available, if aquired,     #
# this would be best ran with a proper stereo camera set up such as the stereoPI       #
# stereo camera designed for RaspberryPI programmable boards                           #
########################################################################################

## Import Modules

In [ ]:
import numpy as np
import cv2 as cv
import os
import imageio
from matplotlib import pyplot as plt
import time

## Read Videos and cut into frames

In [ ]:
def left_ims(pathIn, pathOut):
    
    # Video file
    vidcap = cv.VideoCapture(pathIn)
    success,image = vidcap.read()
    
    # Framerate
    fps = int(vidcap.get(cv.CAP_PROP_FPS))
    length = int(vidcap.get(cv.CAP_PROP_FRAME_COUNT)) # Number of frames

    # Print footage properties
    print('FPS:',fps) # Frames per second
    print('Extracting every {} frames'.format(1))
    print('Total Frames:',length)
    print('Number of Frames Saved:', (length // 1) + 1)
    
    # Iterate over all frames and separate them into individual images
    count = 0
    success = True
    # Limit to length of video
    while count <= length:
        vidcap.set(cv.CAP_PROP_POS_FRAMES,count/2)
        success,image = vidcap.read()
        # Convert to grayscale
        gray = cv.cvtColor(image,cv.COLOR_BGR2GRAY)

        if not success:
            break
        # Save frame as a .jpg with frame number
        cv.imwrite(pathOut+"//left_frame%000d.jpg"%count, gray) 
        count += 1
        
def right_ims(pathIn, pathOut):
    # Follows same process as above for the right video
    vidcap = cv.VideoCapture(pathIn)
    success,image = vidcap.read()
    
    fps = int(vidcap.get(cv.CAP_PROP_FPS))
    length = int(vidcap.get(cv.CAP_PROP_FRAME_COUNT))

    # Print footage properties
    print('FPS:',fps) # Frames per second
    print('Extracting every {} frames'.format(1))
    print('Total Frames:',length)
    print('Number of Frames Saved:', (length // 1) + 1)
    
    count = 0
    success = True
    while count <= length:
        vidcap.set(cv.CAP_PROP_POS_FRAMES,count/2)
        success,image = vidcap.read() 
        gray = cv.cvtColor(image,cv.COLOR_BGR2GRAY)

        if not success:
            break
        cv.imwrite( pathOut + "\\right_frame%000d.jpg" % count, gray)
        count += 1


## Generate Depth Map

In [ ]:
# A function to generate a depth map from stereo frames
# with inputs of file names, number of disparities, block size
# and minimum disparity (minimum depth to check)


def frame_depth(frame_L,frame_R,numDisp=50,blockSz=2,min_disp=10):
    window_size = 2
    # Stereo calibration settings discussed in report, follows pretty
    # closely to the settings needed for the custom depth maps previously
    # generated
    stereo = cv.StereoSGBM_create(minDisparity = min_disp,
                                  numDisparities = numDisp,
                                  blockSize = blockSz,
                                  P1 = 8*3*window_size**2,
                                  P2 = 32*3*window_size**2,
                                  disp12MaxDiff = 10,
                                  uniquenessRatio = 10,
                                  speckleWindowSize = 1,
                                  speckleRange = 2)
    # Generate the map
    disparity = stereo.compute(frame_L,frame_R)
    return disparity

## Save Frames in an Iteratable array

In [ ]:
# Read all frames from a folder and save in a list to be
# iterated over
def load_images(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)

    return images

## Generate Video Depth Map

In [ ]:
def depth_video(input_folder):
    # Define left and right video
    left_vid = input_folder+'left.mp4'
    right_vid = input_folder+'right.mp4'
    
    # Create folders for the left and right frames
    dir = os.path.join(input_folder,'left')
    if not os.path.exists(dir):
        os.mkdir(dir)
    dir = os.path.join(input_folder,'right')
    if not os.path.exists(dir):
        os.mkdir(dir)
        
    # Cut left and right videos into frames, save in new folders
    left_ims(left_vid,input_folder+'/left')
    right_ims(right_vid,input_folder+'/right')

    # Load all images from each folder into arrays
    left_frames = load_images(input_folder+'left')
    right_frames = load_images(input_folder+'right')
    
    # Create disparity array from left and right images
    depth_frames = []
    framesl = len(left_frames)
    framesr = len(right_frames)
    
    # Ensure that the generated disparity video length is
    # only as long as the shortest of the two stereo videos.
    # Ideally they would be the same but this accounts for 
    # error.
    if framesl > framesr:
        length = framesr
    else:
        length = framesl
        
    # Iterate over images and generate depth maps, saving them to a 
    # larger array
    for i in np.arange(length):        
        # Adjust which of these is commented if wanting to see the difference in quality conversion
        #depth_frames.append(frame_depth(left_frames[i],right_frames[i]).astype(np.uint8)) # Supresses warnings but loses quality
        depth_frames.append(frame_depth(left_frames[i],right_frames[i])) # Has lossy conversion but still better quality

    # Create gif
    imageio.mimsave(input_folder+'depth.gif',depth_frames)
    return depth_frames

## Run Function

In [ ]:
# Generate depth video
# NOTE: lossy conversion will occur but it is still better than
# converting to uint8 as suggested. The option is available by
# changing which depth_frames.append line is commented out
# in the iteration above.

disp = depth_video('/Users/justi/Desktop/comp_test/')

## Example Calibration Code

In [ ]:
# This was not applied in practice due to lack of time and equipment
# suitable

In [ ]:
# Import one extra module
import glob

# This task utilizes a 'chess board' pattern as a grid of squares
# with known distances between corners. Analagous to the dot pattern
# explained in the report
def calibration(folder):

    # Define termination criteria
    stop = (cv.TERM_CRITERIA_EPS + cv.TERM_CRITERIA_MAX_ITER, 50, 0.0001)

    # Define corner points by board size
    obj_points = np.zeros((8*8,3),np.float32) # Stanard 8x8 board
    obj_points[:,:2] = np.mgrid[0:8,0:8].T.reshape(-1,2)

    # Define empty lists to store object and image points
    obj_p = [] # Real Space (x,y,z)
    img_p = [] # Image plane (pixels: i,j)

    # Retrieve files of jpg format
    images = glob.glob(folder+'*.jpg')

    for file in images:
        # Read image
        img = cv.imread(file)
        # Convert to grayscale
        gray = cv.cvtColor(img,cv.COLOR_BGR2GRAY)

        # Locate corners
        success,corner = cv.findChessboardCorners(gray,(8,8),None)

        # If they are found, add to object points and image points
        if success == True:
            obj_p.append(obj_points)

            # Refine corners
            ref_corner = cv.cornerSubPix(gray,corner,(11,11),(-1,-1),stop)
            img_p.append(ref_corner)

            # Draw corners
            img = cv.drawChessboardCorners(img,(8,8),ref_corner,success)
            cv.imshow('img',img)
            cv.waitKey(500)

    # Now return a camera matrix, distortion coefficients, rotation and 
    # translation vectors
    success,cam_m,dist_coeff,rot_vec,tran_vec = cv.calibrateCamera(obj_points,
                                                                   img_points,
                                                                  gray.shape[::-1],
                                                                  None,
                                                                  None)
    return cam_m,dist_coeff,rot_vec,tran_vec

## Undistort Frames

In [ ]:
# Run the above code with calibration images to obtain calibration
# matrix to then apply to all frames
"""
camera_matrix = calibration(folder)[0]
dist_coeff = calibration(folder[1])
"""

# Read frames
def undistort(frame_name,camera_matrix,dist_coeff):
    frame = cv.imread(frame_name)
    height,width = frame.shape[:2]
    new_cam_m, image_region = cv.getOptimalNewCameraMatrix(camera_matrix,
                                                          dist_coeff,
                                                          (width,height),
                                                          1,
                                                          (width,height))

    # Remap frames
    x_map,y_map = cv.initUndistortRectifyMap(camera_matrix,
                                            dist_coeff,
                                            new_cam_m
                                            (width,height),
                                            5)
    distortion = cv.remap(frame,x_map,y_map,cv.INTER_LINEAR)

    # Crop the frame
    x,y,w,h = image_region
    # This cropped frame will be straighted and fully undistorted
    distortion = distortion[y:y+h,x:x+w]
    cv.imwrite(frame_name+'_calibrated.jpg',distortion)